In [ ]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import pandas as pd
# import time

# firefox_options = webdriver.FirefoxOptions()
# firefox_options.add_argument('-headless')

# browser = webdriver.Firefox(options=firefox_options)

# start_t = time.time()
# url = 'https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=9'
# browser.get(url)

# dfs = pd.read_html(str(browser.page_source))
# run_t = (time.time() - start_t)/60
# print(f"run time: {run_t:.2f} mins") 

# browser.quit()



In [ ]:
# import pandas as pd
# import time
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import NoSuchElementException, TimeoutException

# try:
#     firefox_options = webdriver.FirefoxOptions()
#     firefox_options.add_argument('-headless')

#     driver = webdriver.Firefox(options=firefox_options)

#     base_url = "https://www.fightmatrix.com/historical-mma-rankings/generated-historical-rankings/?Issue={}&Division={}&Page={}"

#     columns = ["snapshot_periods", "weight_classes", "rank", "name", "country", "points"]
#     data = []

#     weight_classes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16]  # define weightclasses to obtain


#     for division in weight_classes:
#         issue = 135 # needs any arbitrary issue to get full list of issues from the dropdown
#         driver.get(base_url.format(issue, division, 1)) 
#         try:
#             snapshot_dropdown = Select(driver.find_element(By.NAME, "Issue"))
#             snapshot_periods = [option.get_attribute("value") for option in snapshot_dropdown.options if option.get_attribute("value")]
#             max_pages_per_snapshot = 25
#             start_t = time.time()
#             run_t = 0
#             for issue in snapshot_periods:
#                 page = 1
#                 while page <= max_pages_per_snapshot: 
#                     url = base_url.format(issue, division, page)
#                     driver.get(url)
#                     print("Current URL:", url)
#                     try:
#                         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//table[@class='tblRank']//tbody/tr")))
#                         rows = driver.find_elements(By.XPATH, "//tbody/tr[td[@class='tdRank']]")
#                         rows_alt = driver.find_elements(By.XPATH, "//tbody/tr[td[@class='tdRankAlt']]")
#                         combined_rows = rows + rows_alt
#                         print("Number of rows:", len(combined_rows))
#                         if len(combined_rows) == 0:
#                             run_t = (time.time() - start_t)/60 - run_t
#                             print(f"snapshot done: {run_t:.2f} mins")  
#                             break
#                         for row in rows:
#                             rank = row.find_element(By.CLASS_NAME, "tdRank").text
#                             name = row.find_element(By.CLASS_NAME, "sherLink").text
#                             country = row.find_element(By.TAG_NAME, "img").get_attribute("src").split("/")[-1].split(".")[0]
#                             points = row.find_element(By.CLASS_NAME, "tdBar").text
#                             data.append([issue, division, rank, name, country, points])
#                         for row in rows_alt:
#                             rank = row.find_element(By.CLASS_NAME, "tdRankAlt").text
#                             name = row.find_element(By.CLASS_NAME, "sherLink").text
#                             country = row.find_element(By.TAG_NAME, "img").get_attribute("src").split("/")[-1].split(".")[0]
#                             points = row.find_element(By.CLASS_NAME, "tdBar").text
#                             data.append([issue, division, rank, name, country, points])
#                     except NoSuchElementException:
#                         print("NoSuchElementException for current URL")
#                     page += 1   
#         except NoSuchElementException:
#             print("No Issue Element Found")

#     df = pd.DataFrame(data, columns=columns)


#     driver.quit()

#     print(df)
# #except Exception as e:
# #    print(f"process terminated, quitting driver: {str(e)}")
# finally:
#     if 'driver' in locals() or 'driver' in globals():
#         print("quitting driver")
#         driver.quit()

In [ ]:
# from selenium import webdriver
# firefox_options = webdriver.FirefoxOptions()
# firefox_options.add_argument('-headless')
# driver = webdriver.Firefox(options=firefox_options)
# driver.quit()

In [ ]:
# this is to get the list of the actual values corresponding to "issue" or "division".
# also useful to get a full list of each snapshot issue

import requests
from bs4 import BeautifulSoup
import pandas as pd

dummy_site = "https://www.fightmatrix.com/historical-mma-rankings/generated-historical-rankings/?Issue=1&Division=1&Page=1"
html = requests.get(dummy_site).text
soup = BeautifulSoup(html, 'html.parser')
select_iss = soup.find('select', attrs={'name': 'Issue'})
select_div = soup.find('select', attrs={'name': 'Division'})

issue_list = []
div_list = []


for option in select_iss.find_all('option'):
    value = option['value']
    text = option.get_text(strip=True)
    issue_list.append((value, text))

for option in select_div.find_all('option'):
    value = option['value']
    text = option.get_text(strip=True)
    div_list.append((value, text))

issue_df = pd.DataFrame(issue_list, columns=['issue', 'date'])
div_df = pd.DataFrame(div_list, columns=['division', 'weightclass'])


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = "https://www.fightmatrix.com/historical-mma-rankings/generated-historical-rankings/?Issue={}&Division={}&Page={}"
division = [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16] # could also use div_df['weightclass']
issue = issue_df['issue'] # 128 for testing

rank_data = []
fighter_data = []
record_data = []
points_data = []
country_data = []
division_data = []
issue_data = []

for div in division:
    for iss in issue:
        page = 1
        while True:
            url = base_url.format(iss, div, page)
            response = requests.get(url)

            if response.status_code == 200:
                html = response.text

                soup = BeautifulSoup(html, 'html.parser')
                table = soup.find('table', class_='tblRank')

                # important:
                # if page is blank, skip to the next issue (stops it from adding pages to loop through for an issue)
                if len(table.find_all('tr')[1:]) == 0:
                    print(f"no more data for issue {iss}, div {div}.")
                    break

                for row in table.find_all('tr')[1:]:  # skip header 
                    cells = row.find_all(['td', 'a', 'div']) 
                    rank = cells[0].text.strip() if cells[0].text.strip() else None
                    country = cells[2].find('img')['src'].split('/')[-1].split('.')[0]
                    fighter = row.find(class_='sherLink').strong.text.strip()
                    record = cells[4].text.strip() if cells[4].text.strip() else None
                    points = row.find(class_='tdBar').text.strip()

                    rank_data.append(rank)
                    fighter_data.append(fighter)
                    country_data.append(country)
                    record_data.append(record)
                    points_data.append(points)
                    division_data.append(div)
                    issue_data.append(iss)

                time.sleep(0.25) # delay

                print(f"issue {iss}, division {div}, p {page}")
                page += 1
            else:
                print(f"Failed to retrieve the webpage for Issue {iss}, Division {div}.")
                break

        print(f"done: issue {iss} div {div}")

    print(f"done: div {div}")

# Create a Pandas DataFrame
df = pd.DataFrame({
    'Issue': issue_data,
    'Division': division_data,
    'Rank': rank_data,
    'Country': country_data,
    'Fighter': fighter_data,
    'Record': record_data,
    'Points': points_data
})

# Print the DataFrame
print(df)


In [ ]:
# if terminated early:

df = pd.DataFrame({
    'Issue': issue_data,
    'Division': division_data,
    'Rank': rank_data,
    'Country': country_data,
    'Fighter': fighter_data,
    'Record': record_data,
    'Points': points_data
})